In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE244123"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE244123"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE244123.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE244123.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE244123.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Olfactory neuroblastoma mimics molecular subtypes and lineage trajectories of small cell lung cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: olfactory neuroblastoma'], 1: [nan, 'grade: II', 'grade: III', 'grade: normal', 'grade: IV'], 2: [nan, 'Stage: C', 'Stage: A', 'Stage: D', 'Stage: na'], 3: [nan, 'location: brain', 'location: nasal cavity', 'location: Cribriform', 'location: cervical LN'], 4: [nan, 'Sex: F', 'Sex: M'], 5: [nan, 'age: 63', 'age: 69', 'age: 42', 'age: 43', 'age: 76', 'age: 73', 'age: 44', 'age: 54', 'age: 67', 'age: 48']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset

# 1. Gene Expression Data Availability
# This dataset appears to be related to olfactory neuroblastoma and lung cancer
# Based on the title and characteristics, it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait identification - Lung Cancer
# Looking at sample characteristics, there's information about tumor/cancer grades (key 1)
# We can infer cancer presence from the 'grade' field
trait_row = 1

# Function to convert trait data
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Normal samples considered as control (0), all cancer grades as cases (1)
    if 'normal' in value.lower():
        return 0
    elif any(grade in value.lower() for grade in ['i', 'ii', 'iii', 'iv']):
        return 1
    else:
        return None

# 2.2 Age data is available in key 5
age_row = 5

def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to float for continuous age
        return float(value)
    except:
        return None

# 2.3 Gender data is available in key 4
gender_row = 4

def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: female=0, male=1
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
# Determine trait data availability
is_trait_available = trait_row is not None

# Perform initial validation and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Assuming clinical_data is available from a previous step
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM7807444': [nan, nan, nan], 'GSM7807445': [1.0, 63.0, 0.0], 'GSM7807446': [1.0, 63.0, 0.0], 'GSM7807447': [1.0, 63.0, 0.0], 'GSM7807448': [1.0, 63.0, 0.0], 'GSM7807449': [1.0, 63.0, 0.0], 'GSM7807450': [1.0, 63.0, 0.0], 'GSM7807451': [1.0, 63.0, 0.0], 'GSM7807452': [1.0, 63.0, 0.0], 'GSM7807453': [1.0, 63.0, 0.0], 'GSM7807454': [1.0, 69.0, 0.0], 'GSM7807455': [1.0, 69.0, 0.0], 'GSM7807456': [1.0, 69.0, 0.0], 'GSM7807457': [1.0, 69.0, 0.0], 'GSM7807458': [1.0, 42.0, 1.0], 'GSM7807459': [1.0, 42.0, 1.0], 'GSM7807460': [1.0, 42.0, 1.0], 'GSM7807461': [1.0, 43.0, 0.0], 'GSM7807462': [1.0, 43.0, 0.0], 'GSM7807463': [1.0, 43.0, 0.0], 'GSM7807464': [1.0, 43.0, 0.0], 'GSM7807465': [1.0, 43.0, 0.0], 'GSM7807466': [1.0, 43.0, 0.0], 'GSM7807467': [1.0, 76.0, 1.0], 'GSM7807468': [1.0, 76.0, 1.0], 'GSM7807469': [1.0, 76.0, 1.0], 'GSM7807470': [1.0, 76.0, 1.0], 'GSM7807471': [1.0, 76.0, 1.0], 'GSM7807472': [1.0, 76.0, 1.0], 'GSM7807473': [1.0, 76.0, 1.0], 'GSM780747

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AADAT', 'AAGAB',
       'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are standard human gene symbols that don't require mapping
# A1BG = Alpha-1-B Glycoprotein
# A1CF = APOBEC1 Complementation Factor
# A2M = Alpha-2-Macroglobulin
# etc.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
# Need to load the saved clinical data
clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from olfactory neuroblastoma patients relevant to lung cancer research"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")

Shape of normalized gene data: (18669, 96)


Saved normalized gene data to ../../output/preprocess/Lung_Cancer/gene_data/GSE244123.csv
Shape of linked data: (96, 18672)


Shape of linked data after handling missing values: (95, 18672)
For the feature 'Lung_Cancer', the least common label is '0.0' with 5 occurrences. This represents 5.26% of the dataset.
The distribution of the feature 'Lung_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 44.0
  50% (Median): 63.0
  75%: 69.0
Min: 42.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 46 occurrences. This represents 48.42% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Lung_Cancer/GSE244123.csv
